<a href="https://colab.research.google.com/github/siddtheshah/vc_modeling/blob/master/regression_targets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

!pip install cityhash
import cityhash
from copy import deepcopy

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
file_names = os.listdir("/content/gdrive/My Drive/vc_modeling/data/crunchbase_bulk_export/")
df_names = [x[:-4] for x in file_names]
print(df_names)

dfs = [pd.read_csv("/content/gdrive/My Drive/vc_modeling/data/crunchbase_bulk_export/"+x) for x in file_names]
df_dict = dict(zip(df_names, dfs))
print(df_dict.keys())

['category_groups', 'funding_rounds', 'people', 'checksum', 'people_descriptions', 'investors', 'organization_descriptions', 'investment_partners', 'event_appearances', 'organizations', 'org_parents', 'jobs', 'acquisitions', 'funds', 'ipos', 'degrees', 'investments', 'events']
dict_keys(['category_groups', 'funding_rounds', 'people', 'checksum', 'people_descriptions', 'investors', 'organization_descriptions', 'investment_partners', 'event_appearances', 'organizations', 'org_parents', 'jobs', 'acquisitions', 'funds', 'ipos', 'degrees', 'investments', 'events'])


In [4]:
df_dict["funding_rounds"][:200]

,uuid,name,type,permalink,cb_url,rank,created_at,updated_at,country_code,state_code,region,city,investment_type,announced_on,raised_amount_usd,raised_amount,raised_amount_currency_code,post_money_valuation_usd,post_money_valuation,post_money_valuation_currency_code,investor_count,org_uuid,org_name,lead_investor_uuids
0,8a945939-18e0-cc9d-27b9-bf33817b2818,Angel Round - Facebook,funding_round,facebook-angel--8a945939,https://www.crunchbase.com/funding_round/faceb...,143236.0,2007-05-27 06:08:18,2018-02-12 23:05:39,USA,CA,California,Menlo Park,angel,2004-09-01,500000.0,500000.0,USD,NaN,NaN,USD,4.0,df662812-7f97-0b43-9d3e-12f64f504fbb,Facebook,3f47be49-2e32-8118-01a0-31685a4d0fd7
1,d950d7a5-79ff-fb93-ca87-13386b0e2feb,Series A - Facebook,funding_round,facebook-series-a--d950d7a5,https://www.crunchbase.com/funding_round/faceb...,145939.0,2007-05-27 06:09:10,2018-02-12 23:52:16,USA,CA,California,Menlo Park,series_a,2005-05-01,12700000.0,12700000.0,USD,98000000.0,98000000.0,USD,4.0,df662812-7f97-0b43-9d3e-12f64f504fbb,Facebook,b08efc27-da40-505a-6f9d-c9e14247bf36
2,6fae3958-a001-27c0-fb7e-666266aedd78,Series B - Facebook,funding_round,facebook-series-b--6fae3958,https://www.crunchbase.com/funding_round/faceb...,273250.0,2007-05-27 06:09:36,2018-02-12 23:30:46,USA,CA,California,Menlo Park,series_b,2006-04-01,27500000.0,27500000.0,USD,502500000.0,502500000.0,USD,5.0,df662812-7f97-0b43-9d3e-12f64f504fbb,Facebook,"e2006571-6b7a-e477-002a-f7014f48a7e3,8d5c7e48-..."
3,bcd5a63d-ed99-6963-0dd2-e36f6582f846,Series B - Photobucket,funding_round,photobucket-series-b--bcd5a63d,https://www.crunchbase.com/funding_round/photo...,149479.0,2007-05-29 11:05:59,2018-02-12 23:27:36,USA,CO,Colorado,Denver,series_b,2006-05-01,10500000.0,10500000.0,USD,NaN,NaN,USD,2.0,f53cb4de-236e-0b1b-dee8-7104a8b018f9,Photobucket,NaN
4,60e6afd9-1215-465a-dd17-0ed600d4e29b,Series A - Geni,funding_round,geni-series-a--60e6afd9,https://www.crunchbase.com/funding_round/geni-...,313010.0,2007-05-31 20:19:28,2018-02-12 23:41:29,USA,CA,California,West Hollywood,series_a,2007-01-17,NaN,NaN,NaN,10000000.0,10000000.0,USD,1.0,4111dc8b-c0df-2d24-ed33-30cd137b3098,Geni,fb2f8884-ec07-895a-48d7-d9a9d4d7175c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,45e3988d-85dc-d98e-2738-00ecd8958cc4,Series B - Going,funding_round,going-series-b--45e3988d,https://www.crunchbase.com/funding_round/going...,320247.0,2007-07-25 10:43:40,2018-02-12 23:47:25,USA,MA,Massachusetts,Boston,series_b,2007-07-25,5000000.0,5000000.0,USD,NaN,NaN,USD,2.0,e95cf6be-d512-3d4a-0a28-2a1e7f46c20b,Going,"34a9bb66-0984-8de0-e7fb-13e976b4a135,b51ff078-..."
196,5c3c2595-5bd3-8cd8-e254-e6a877cca6bc,Series A - eBuddy,funding_round,ebuddy-series-a--5c3c2595,https://www.crunchbase.com/funding_round/ebudd...,96837.0,2007-07-25 11:54:44,2018-02-12 22:50:11,USA,CA,California,San Francisco,series_a,2006-10-26,6338871.0,5000000.0,EUR,NaN,NaN,USD,1.0,84a47fd6-8612-e375-0101-70e9e13cb56a,eBuddy,1371e6ae-921d-4346-5815-788c87e01e15
197,e514c980-5fd2-f154-65cc-d3e3f0a7d2d5,Seed Round - Jaxtr,funding_round,jaxtr-seed--e514c980,https://www.crunchbase.com/funding_round/jaxtr...,277460.0,2007-07-26 06:01:01,2020-06-19 08:24:19,USA,CA,California,Menlo Park,seed,2007-07-01,1500000.0,1500000.0,USD,NaN,NaN,USD,4.0,edbfbb52-4487-1d7c-1960-7d95f2e6809e,Jaxtr,NaN
198,27ba6606-2845-1abd-9d49-e90c7be92eb2,Series A - Kiptronic,funding_round,kiptronic-series-a--27ba6606,https://www.crunchbase.com/funding_round/kiptr...,197298.0,2007-07-26 06:07:04,2018-02-12 23:43:33,USA,CA,California,San Francisco,series_a,2006-12-01,4000000.0,4000000.0,USD,NaN,NaN,USD,2.0,0fc2d002-8102-1ecc-59dc-06ef55f6c7f0,Kiptronic,"1a3fa9b2-a5a7-0b47-15af-5984cbb0b9a9,fcc72760-..."


In [38]:
def maybe_add_funding_estimates(row):
  # print(row)
  # print("Row printed")
  # print(np.isnan(row['post_money_valuation_usd']))
  if np.isnan(row['post_money_valuation_usd']) and not np.isnan(row['raised_amount_usd']):
    if row['raised_amount_usd'] < 1:
      row['post_money_valuation_usd'] = 10000
    else:
      # print("Replaced: ", row['post_money_valuation_usd'], " ", row['raised_amount_usd']*3)
      row['post_money_valuation_usd'] = row['raised_amount_usd']*5
  return row['post_money_valuation_usd']

df = pd.DataFrame(list(zip([np.nan, np.nan], ['1000', '1000'])) ,columns =['post_money_valuation_usd', 'raised_amount_usd']).astype(np.float32)
df['post_money_valuation_usd'] = df.apply(maybe_add_funding_estimates, axis=1)

In [46]:
imputed_funding_rounds = deepcopy(df_dict["funding_rounds"])
# print(imputed_funding_rounds)
imputed_funding_rounds['post_money_valuation_usd'] = imputed_funding_rounds.apply(maybe_add_funding_estimates, axis=1)
# print(imputed_funding_rounds[['post_money_valuation_usd', 'raised_amount_usd']])

In [47]:
print(imputed_funding_rounds.head())

funding_data = imputed_funding_rounds[['org_uuid', 'announced_on', 'post_money_valuation_usd']].rename(
    columns={"announced_on": "date", "post_money_valuation_usd": "valuation"})

acquisitions_data = deepcopy(df_dict["acquisitions"][['acquiree_uuid', 'acquired_on', 'price_usd']]).rename(
    columns={"acquiree_uuid": "org_uuid", "acquired_on": "date", "price_usd": "valuation"})
ipos_data = deepcopy(df_dict["ipos"][['org_uuid', 'went_public_on', 'valuation_price_usd']]).rename(
    columns={"went_public_on": "date", "valuation_price_usd": "valuation"})

types = df_dict["acquisitions"][['type']].values
acquisition_types = df_dict["acquisitions"][['acquisition_type']].values
acquisitions_data['event'] = [types[x][0] if not isinstance(acquisition_types[x][0], str) 
                              else acquisition_types[x][0] for x in range(len(acquisitions_data))]
funding_data['event'] = pd.Series([x[0] for x in df_dict["funding_rounds"]['name'].str.split(" - ")])
ipos_data['event'] = df_dict['ipos']['stock_exchange_symbol']



                                   uuid  ...                                lead_investor_uuids
0  8a945939-18e0-cc9d-27b9-bf33817b2818  ...               3f47be49-2e32-8118-01a0-31685a4d0fd7
1  d950d7a5-79ff-fb93-ca87-13386b0e2feb  ...               b08efc27-da40-505a-6f9d-c9e14247bf36
2  6fae3958-a001-27c0-fb7e-666266aedd78  ...  e2006571-6b7a-e477-002a-f7014f48a7e3,8d5c7e48-...
3  bcd5a63d-ed99-6963-0dd2-e36f6582f846  ...                                                NaN
4  60e6afd9-1215-465a-dd17-0ed600d4e29b  ...               fb2f8884-ec07-895a-48d7-d9a9d4d7175c

[5 rows x 24 columns]


In [48]:
funding_data.head()

,org_uuid,date,valuation,event
0,df662812-7f97-0b43-9d3e-12f64f504fbb,2004-09-01,2500000.0,Angel Round
1,df662812-7f97-0b43-9d3e-12f64f504fbb,2005-05-01,98000000.0,Series A
2,df662812-7f97-0b43-9d3e-12f64f504fbb,2006-04-01,502500000.0,Series B
3,f53cb4de-236e-0b1b-dee8-7104a8b018f9,2006-05-01,52500000.0,Series B
4,4111dc8b-c0df-2d24-ed33-30cd137b3098,2007-01-17,10000000.0,Series A


In [52]:
timelines = pd.concat([funding_data, acquisitions_data, ipos_data])

# Normalize dates

timelines.sort_values(by=['org_uuid', 'date'], inplace=True)
timelines.reset_index(inplace=True, drop=True)

timelines.set_index(['org_uuid'])
grouped_timelines = timelines.groupby(['org_uuid'])

In [53]:
grouped_timelines.head()

,org_uuid,date,valuation,event
0,00000aa4-ba42-9b68-a9c3-040c9f3bf9b9,2013-02-25,NaN,lbo
1,00000aa4-ba42-9b68-a9c3-040c9f3bf9b9,2017-05-12,NaN,acquisition
2,00000aa4-ba42-9b68-a9c3-040c9f3bf9b9,2017-07-17,413036820.0,Venture Round
3,000014da-0c46-b9cb-0941-3a93c027b119,2016-11-01,NaN,Pre Seed Round
4,000014da-0c46-b9cb-0941-3a93c027b119,2017-01-01,NaN,Grant
...,...,...,...,...
470726,ffffabce-6d4a-b3d1-13c0-4e90cedf5270,2016-05-15,1050000.0,Seed Round
470727,ffffabce-6d4a-b3d1-13c0-4e90cedf5270,2017-02-09,3500000.0,Seed Round
470729,ffffae3e-fc62-68e7-7b49-f6843c3662f4,2015-04-22,NaN,acquisition
470730,ffffae3e-fc62-68e7-7b49-f6843c3662f4,2017-11-07,NaN,acquisition


In [54]:
inspect_id = '000014da-0c46-b9cb-0941-3a93c027b119'
organizations = df_dict['organizations']
funding_data[funding_data['org_uuid'] == inspect_id].head()
# organizations[organizations['uuid'] == inspect_id].head()

,org_uuid,date,valuation,event
162921,000014da-0c46-b9cb-0941-3a93c027b119,2016-11-01,NaN,Pre Seed Round
162922,000014da-0c46-b9cb-0941-3a93c027b119,2017-01-01,NaN,Grant
176451,000014da-0c46-b9cb-0941-3a93c027b119,2017-06-15,NaN,Grant
176816,000014da-0c46-b9cb-0941-3a93c027b119,2017-09-07,NaN,Convertible Note
181152,000014da-0c46-b9cb-0941-3a93c027b119,2018-02-23,NaN,Pre Seed Round


In [55]:
import datetime
company_timelines = {}

milli_per_day = 86400  # Used for converting Unix time
dropped = 0

for company_id in timelines['org_uuid']:
  timeline = grouped_timelines.get_group(company_id)
  timeline = timeline[timeline['valuation'].notnull()]               # We only care about times at which there is a valuation of the company.
  if timeline.empty:
    dropped += 1
    continue
  
  timeline = timeline.copy()
  timeline.reset_index(inplace=True, drop=True)

  day_zero = datetime.datetime.strptime(timeline['date'].iloc[0], '%Y-%m-%d').timestamp()
  parsed_dates = pd.Series([datetime.datetime.strptime(x, '%Y-%m-%d').timestamp() for x in timeline['date']])
  norm_dates = (parsed_dates - day_zero)/milli_per_day
  timeline['norm_dates'] = norm_dates
  company_timelines[company_id] = timeline

print("Retained: ", len(company_timelines))
print("Dropped: ", dropped)

Retained:  144569
Dropped:  152322


In [56]:
print(company_timelines['45e3988d-85dc-d98e-2738-00ecd8958cc4'])

KeyError: ignored

In [57]:
for company_id, timeline in company_timelines.items():
  initial_valuation = timeline['valuation'].iloc[0]
  timeline['initial_val'] = initial_valuation.tolist()
  norm_valuations = pd.Series([np.log(value/initial_valuation) for value in timeline['valuation']])
  timeline['norm_valuations'] = norm_valuations

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in double_scalars
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


In [58]:
from scipy import interpolate
from google.colab import files

def get_valuation_interpolations(timelines, day):
  company_hashes = []
  regression_targets = []
  initial_valuations = []
  for company, timeline in timelines.items():
    x = timeline['norm_dates'].tolist()
    y = timeline['norm_valuations'].tolist()

    if len(x) < 2 or len(y) < 2:
      regression_targets.append(0)     # Single point timeline. Leave decision to drop for downstream.
      continue
    
    if day > x[-1]:
      regression_targets.append(y[-1])  # Don't extrapolate the data at all.
      continue

    interpolator = interpolate.interp1d(x, y)
    regression_target = interpolator(day)

    company_hashes.append(cityhash.CityHash64(company))
    regression_targets.append(regression_target)
    initial_valuations.append(timeline['initial_val'].tolist()[0])

  return pd.DataFrame(list(zip(company_hashes, initial_valuations, regression_targets)), columns=["hash", "initial_valuation", "log_valuation_factor"])


In [59]:
for day in [200, 500, 1000, 2000]:
  frame = get_valuation_interpolations(company_timelines, day)
  print(np.shape(frame))
  frame.to_csv('/' + str(day) + '.csv')


(49489, 3)
(38340, 3)
(23740, 3)
(9156, 3)
